***GENERATED CODE FOR carregression PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Car_id', 'transformation_label': 'String Indexer'}], 'feature': 'Car_id', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'C_CND_000001', 'max': 'C_CND_000500', 'missing': '0', 'distinct': '500'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Car_id'}, {'feature_label': 'Car_id', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Car_id')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'Date', 'transformation_label': 'Extract Date'}], 'feature': 'Date', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'Date'}, {'feature_label': 'Date', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('Date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Customer_Name', 'transformation_label': 'String Indexer'}], 'feature': 'Customer_Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Aaliyah', 'max': 'Zoey', 'missing': '0', 'distinct': '443'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Customer_Name'}, {'feature_label': 'Customer_Name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Customer_Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Female', 'max': 'Male', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Dealer_Name', 'transformation_label': 'String Indexer'}], 'feature': 'Dealer_Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': "Buddy Storbeck's Diesel Service Inc", 'max': 'U-Haul CO', 'missing': '0', 'distinct': '28'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Dealer_Name'}, {'feature_label': 'Dealer_Name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Dealer_Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Company', 'transformation_label': 'String Indexer'}], 'feature': 'Company', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Acura', 'max': 'Volvo', 'missing': '0', 'distinct': '30'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Company'}, {'feature_label': 'Company', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Company')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Model', 'transformation_label': 'String Indexer'}], 'feature': 'Model', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '3-Sep', 'max': 'Xterra', 'missing': '0', 'distinct': '136'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Model'}, {'feature_label': 'Model', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Model')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Engine', 'transformation_label': 'String Indexer'}], 'feature': 'Engine', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'DoubleÂ\xa0Overhead Camshaft', 'max': 'Overhead Camshaft', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Engine'}, {'feature_label': 'Engine', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Engine')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Transmission', 'transformation_label': 'String Indexer'}], 'feature': 'Transmission', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Auto', 'max': 'Manual', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Transmission'}, {'feature_label': 'Transmission', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Transmission')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Color', 'transformation_label': 'String Indexer'}], 'feature': 'Color', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Black', 'max': 'Red', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Color'}, {'feature_label': 'Color', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Color')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Dealer_No ', 'transformation_label': 'String Indexer'}], 'feature': 'Dealer_No ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '06457-3834', 'max': '99301-3882', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Dealer_No '}, {'feature_label': 'Dealer_No ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Dealer_No ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Body_Style', 'transformation_label': 'String Indexer'}], 'feature': 'Body_Style', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Hardtop', 'max': 'Sedan', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Body_Style'}, {'feature_label': 'Body_Style', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Body_Style')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Dealer_Region', 'transformation_label': 'String Indexer'}], 'feature': 'Dealer_Region', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Aurora', 'max': 'Scottsdale', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Dealer_Region'}, {'feature_label': 'Dealer_Region', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Dealer_Region')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run carregressionHooks.ipynb
try:
	#sourcePreExecutionHook()

	carsalescustomer = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/car_sales_customer.csv', 'filename': 'car_sales_customer.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(carsalescustomer)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run carregressionHooks.ipynb
try:
	#transformationPreExecutionHook()

	carregressionautofe = TransformationMain.run(carsalescustomer,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Car_id", "transformation_label": "String Indexer"}], "feature": "Car_id", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "C_CND_000001", "max": "C_CND_000500", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Car_id"}, {"transformationsData": [{"feature_label": "Date", "transformation_label": "Extract Date"}], "feature": "Date", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "Date"}, {"transformationsData": [{"feature_label": "Customer_Name", "transformation_label": "String Indexer"}], "feature": "Customer_Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Aaliyah", "max": "Zoey", "missing": "0", "distinct": "443"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Customer_Name"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Female", "max": "Male", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Annual_Income", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "864606.3", "stddev": "693235.45", "min": "13500", "max": "4060000", "missing": "0"}, "updatedLabel": "Annual_Income"}, {"transformationsData": [{"feature_label": "Dealer_Name", "transformation_label": "String Indexer"}], "feature": "Dealer_Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Buddy Storbeck's Diesel Service Inc", "max": "U-Haul CO", "missing": "0", "distinct": "28"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Dealer_Name"}, {"transformationsData": [{"feature_label": "Company", "transformation_label": "String Indexer"}], "feature": "Company", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Acura", "max": "Volvo", "missing": "0", "distinct": "30"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Company"}, {"transformationsData": [{"feature_label": "Model", "transformation_label": "String Indexer"}], "feature": "Model", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "3-Sep", "max": "Xterra", "missing": "0", "distinct": "136"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Model"}, {"transformationsData": [{"feature_label": "Engine", "transformation_label": "String Indexer"}], "feature": "Engine", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Double\u00c2\u00a0Overhead Camshaft", "max": "Overhead Camshaft", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Engine"}, {"transformationsData": [{"feature_label": "Transmission", "transformation_label": "String Indexer"}], "feature": "Transmission", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Auto", "max": "Manual", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Transmission"}, {"transformationsData": [{"feature_label": "Color", "transformation_label": "String Indexer"}], "feature": "Color", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Black", "max": "Red", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Color"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Price ", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "28249.26", "stddev": "14413.04", "min": "9000", "max": "85600", "missing": "0"}, "updatedLabel": "Price "}, {"transformationsData": [{"feature_label": "Dealer_No ", "transformation_label": "String Indexer"}], "feature": "Dealer_No ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "06457-3834", "max": "99301-3882", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Dealer_No "}, {"transformationsData": [{"feature_label": "Body_Style", "transformation_label": "String Indexer"}], "feature": "Body_Style", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Hardtop", "max": "Sedan", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Body_Style"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Phone", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "7511319.23", "stddev": "874680.83", "min": "6000458", "max": "8997806", "missing": "0"}, "updatedLabel": "Phone"}, {"transformationsData": [{"feature_label": "Dealer_Region", "transformation_label": "String Indexer"}], "feature": "Dealer_Region", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Aurora", "max": "Scottsdale", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Dealer_Region"}]}))

	#transformationPostExecutionHook(carregressionautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run carregressionHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(carregressionautofe, ["Annual_Income", "Phone", "Car_id_stringindexer", "Date_dayofmonth", "Date_month", "Date_year", "Customer_Name_stringindexer", "Gender_stringindexer", "Dealer_Name_stringindexer", "Company_stringindexer", "Model_stringindexer", "Engine_stringindexer", "Transmission_stringindexer", "Color_stringindexer", "Dealer_No _stringindexer", "Body_Style_stringindexer", "Dealer_Region_stringindexer"], "Price ")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

